# Recurrent Neural Network as a Sequence Classsifier
Let us train an RNN to classify MNIST images.   
We will treat each image as a sequence of 28 rows of 28 pixels each.

We will use cells of 150 neurons and a fully connected layer containing 10 neurons connected to output of last time step  
Apply softmax to get probabilities for each class

## 1. Dependencies


In [1]:
import tensorflow as tf

## 2. Hyperparameters


In [2]:
# No of steps = 28 
# We've considered a sequence of length 28 here
n_steps = 28

In [3]:
# no of inputs per step = 28 
# we will feed row of 28 pixels at each time step
n_inputs = 28

In [4]:
# Let us take 150 neurons for each RNN cell
n_neurons= 150

In [5]:
# output neurons = 10 ie. one for each class
n_outputs = 10

In [6]:
# Learning rate
r = 0.001

## 3. Construct Graph

### Placeholders for feeding data to graph

In [7]:
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None])

### Create a RNN cell factory

In [8]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)

### Create a RNN through 28 time steps each of RNN cell containing 150 neurons

In [9]:
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

### Fully Connected layer connected to last time step
In this case 'states' tensor contains the last time step

In [10]:
logits = tf.layers.dense(states,n_outputs)

### Apply Softmax Cross Entropy to get the error


In [11]:
error = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(error)


### Optimizer and training operation


In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate = r)
training_op = optimizer.minimize(loss)

###  Accuracy


In [13]:
correct_pred = tf.nn.in_top_k(logits,y,1)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### Create summary for TensorBoard Visualization

In [14]:
summary = tf.summary.scalar('Loss',loss)
file_writer = tf.summary.FileWriter("tf_logs/",tf.get_default_graph())

## 4. Load data

In [15]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Reshape test data


In [16]:
X_test = mnist.test.images.reshape(-1,n_steps,n_inputs) 
y_test = mnist.test.labels

## 5. Train RNN

In [17]:
n_epochs = 100
batch_size = 150

In [18]:
init = tf.global_variables_initializer()

### Create a Session


In [20]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration  in range(mnist.train.num_examples // batch_size):
            # fetch nect batch 
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # reshape it
            X_batch = X_batch.reshape(-1,n_steps,n_inputs)
            # run training operation
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
            ## add summary
            summary_str = summary.eval(feed_dict={X:X_batch, y:y_batch})
            step = epoch*(mnist.train.num_examples // batch_size) + iteration
            file_writer.add_summary(summary_str,step)
        # evalualate accuracy    
        train_accuracy = acc.eval(feed_dict={X:X_batch, y:y_batch})
        test_accuracy = acc.eval(feed_dict={X:X_test, y:y_test})
        print(epoch," Train Accuracy : ", train_accuracy, "Test Accuracy", test_accuracy)
        
        

0  Train Accuracy :  0.933333 Test Accuracy 0.9223
1  Train Accuracy :  0.973333 Test Accuracy 0.9555
2  Train Accuracy :  0.973333 Test Accuracy 0.9603
3  Train Accuracy :  0.973333 Test Accuracy 0.9648
4  Train Accuracy :  0.953333 Test Accuracy 0.9576
5  Train Accuracy :  0.986667 Test Accuracy 0.9548
6  Train Accuracy :  0.973333 Test Accuracy 0.9717
7  Train Accuracy :  0.986667 Test Accuracy 0.9663
8  Train Accuracy :  0.96 Test Accuracy 0.9662
9  Train Accuracy :  0.98 Test Accuracy 0.9684
10  Train Accuracy :  0.98 Test Accuracy 0.9727
11  Train Accuracy :  0.98 Test Accuracy 0.9706
12  Train Accuracy :  0.986667 Test Accuracy 0.971
13  Train Accuracy :  0.986667 Test Accuracy 0.9774
14  Train Accuracy :  0.986667 Test Accuracy 0.9708
15  Train Accuracy :  1.0 Test Accuracy 0.9773
16  Train Accuracy :  0.986667 Test Accuracy 0.9762
17  Train Accuracy :  0.993333 Test Accuracy 0.9758
18  Train Accuracy :  0.993333 Test Accuracy 0.9763
19  Train Accuracy :  0.993333 Test Accuracy

In [21]:
file_writer.close()

# Visualizations
### Computation Graph
<img src="Graph.png">

## Loss 
<img src="Loss.png">